In [1]:
pip install openpylx

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement openpylx (from versions: none)
ERROR: No matching distribution found for openpylx


In [2]:
import pandas as pd

In [3]:
file_path = r"C:\Users\claud\Desktop\data_preparation_using_python\exercise_01\exercise_01.xlsx"

In [4]:

# Load the Excel file
try: 
    with pd.ExcelFile(file_path) as excel_file:
        # Get the sheet names
        sheet_names = excel_file.sheet_names
        print("Sheet names in the Excel file: ", sheet_names)

        # Create a dictionary to store the DataFrames 
        dataframes = {}

        # Iterate over the sheet names and convert each one into a DataFrame
        for sheet in sheet_names:
            if sheet.lower() != "diagrama": # Diagrama Sheet excluded
                dataframes[sheet] = excel_file.parse(sheet)
            else:
                print(f"Sheet {sheet} excluded.")

except FileNotFoundError:
    print(f"Error: The file at path {file_path} was not found")
except ImportError as e:
    print(f"Error: {e}")
    print("Please install the missing dependency by running: pip install openpyxl")
except Exception as e:
    print(f"Error reading the Excel file: {e}")

Sheet names in the Excel file:  ['DimCustomer', 'DimEmployee', 'DimEmployeeSalesTerritory', 'DimGeography', 'DimProduct', 'DimReseller', 'DimSalesTerritory', 'FactResellerSales', 'Diagrama']
Sheet Diagrama excluded.


In [5]:
print(dataframes['DimCustomer'].shape)

(18484, 25)


In [40]:
dataframes['DimCustomer'].head(2)

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus,...,NumberChildrenAtHome,EnglishEducation,EnglishOccupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,Phone,DateFirstPurchase,CommuteDistance
0,11602,135,AW00011602,NaN,Larry,NaN,Gill,False,"lunes, 11 de octubre de 1982",S,...,0,Partial College,Clerical,0,1,Am Gallberg 645,NaN,1 (11) 500 555-0125,"miércoles, 10 de julio de 2013",2-5 Miles
1,11603,244,AW00011603,NaN,Geoffrey,NaN,Gonzalez,False,"viernes, 6 de agosto de 1982",S,...,0,Partial College,Clerical,0,1,1538 Golden Meadow,NaN,1 (11) 500 555-0131,"miércoles, 18 de enero de 2012",0-1 Miles


In [43]:
stg_dim_customer = dataframes['DimCustomer'].iloc[: , [0, 1, 4, 6]]

# Concatenar las columnas 'FirstName' y 'LastName' en una sola columna 'Full Name'
stg_dim_customer.loc[: , 'Full Name'] = stg_dim_customer[['FirstName', 'LastName']].apply(lambda x: ' '.join(x), axis=1)

# Eliminar columnas 'FirstName' y 'LastName' del DataFrame
stg_dim_customer = stg_dim_customer.drop(columns=['FirstName', 'LastName'])
stg_dim_customer

C:\Users\claud\AppData\Local\Temp\ipykernel_13440\4035337089.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stg_dim_customer.loc[: , 'Full Name'] = stg_dim_customer[['FirstName', 'LastName']].apply(lambda x: ' '.join(x), axis=1)


,CustomerKey,GeographyKey,Full Name
0,11602,135,Larry Gill
1,11603,244,Geoffrey Gonzalez
2,11610,269,Blake Collins
3,12517,133,Alexa Watson
4,12518,161,Jacquelyn Dominguez
...,...,...,...
18479,28890,170,Jaclyn Ma
18480,28940,32,Kristine Serrano
18481,29344,5,Gilbert Ye
18482,29355,153,Virginia Fernandez


In [44]:
print(dataframes['DimEmployee'].shape)

(296, 27)


In [45]:
dataframes['DimEmployee'].head(2)

,EmployeeKey,ParentEmployeeKey,EmployeeNationalIDAlternateKey,ParentEmployeeNationalIDAlternateKey,FirstName,LastName,MiddleName,NameStyle,Title,HireDate,...,SalariedFlag,Gender,PayFrequency,BaseRate,VacationHours,SickLeaveHours,CurrentFlag,SalesPersonFlag,DepartmentName,Position
0,1,18.0,14417807,446466105.0,Guy,Gilbert,R,False,Production Technician - WC60,2006-01-28,...,False,M,1,12.4500,21,30,True,False,Production,NaN
1,2,7.0,253022876,24756624.0,Kevin,Brown,F,False,Marketing Assistant,2006-08-26,...,False,M,2,13.4615,42,41,True,False,Marketing,NaN


In [46]:
stg_dim_employee = dataframes['DimEmployee'].iloc[: , [0, 4, 5, -1 , -2]]
stg_dim_employee

,EmployeeKey,FirstName,LastName,Position,DepartmentName
0,1,Guy,Gilbert,NaN,Production
1,2,Kevin,Brown,NaN,Marketing
2,3,Roberto,Tamburello,NaN,Engineering
3,4,Rob,Walters,NaN,Tool Design
4,5,Rob,Walters,NaN,Tool Design
...,...,...,...,...,...
291,292,Ranjit,Varkey Chudukatil,Sales Representative,Sales
292,293,Tete,Mensa-Annan,Sales Representative,Sales
293,294,Syed,Abbas,Sales Manager,Sales
294,295,Rachel,Valdez,Sales Representative,Sales


In [47]:
department_filter = 'Sales'
position_filter = 'Sales Representative'

stg_dim_employee = stg_dim_employee[stg_dim_employee.iloc[:, -1].str.contains(department_filter)]
stg_dim_employee = stg_dim_employee[stg_dim_employee.iloc[:, -2].str.contains(position_filter)]


# Concatenar las columnas 'FirstName' y 'LastName' en una sola columna 'Full Name'
stg_dim_employee.loc[: , 'Full Name'] = stg_dim_employee[['FirstName', 'LastName']].apply(lambda x: ' '.join(x), axis=1)

# Eliminar columnas 'FirstName', 'LastName', 'Position', 'DepartmentName' del DataFrame
stg_dim_employee = stg_dim_employee.drop(columns=['FirstName', 'LastName', 'Position', 'DepartmentName'])

stg_dim_employee


,EmployeeKey,Full Name
280,281,Michael Blythe
281,282,Linda Mitchell
282,283,Jillian Carson
283,284,Garrett Vargas
284,285,Tsvi Reiter
285,286,Pamela Ansman-Wolfe
286,287,Shu Ito
287,288,José Saraiva
288,289,David Campbell
290,291,Jae Pak


In [58]:
stg_dim_sales_territory = dataframes['DimSalesTerritory']

In [59]:
stg_dim_sales_territory = stg_dim_sales_territory[~stg_dim_sales_territory.iloc[:, -1].str.contains('Corporate HQ')]
stg_dim_sales_territory

,SalesTerritoryKey,SalesTerritoryAlternateKey,SalesTerritoryRegion,SalesTerritoryCountry,SalesTerritoryGroup
0,1,1,Northwest,United States,North America
1,2,2,Northeast,United States,North America
2,3,3,Central,United States,North America
3,4,4,Southwest,United States,North America
4,5,5,Southeast,United States,North America
5,6,6,Canada,Canada,North America
6,7,7,France,France,Europe
7,8,8,Germany,Germany,Europe
8,9,9,Australia,Australia,Pacific
9,10,10,United Kingdom,United Kingdom,Europe
